# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession 
from pyspark.sql.types import IntegerType, DoubleType 
from pyspark.sql.functions import col, asc, desc, udf, to_date
import datetime as dt

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here


In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [2]:


spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


#### Immigration Data Cleanup

In [3]:
#write to parquet

df_spark=spark.read.parquet("sas_data")
df_spark.printSchema


<bound method DataFrame.printSchema of DataFrame[cicid: double, i94yr: double, i94mon: double, i94cit: double, i94res: double, i94port: string, arrdate: double, i94mode: double, i94addr: string, depdate: double, i94bir: double, i94visa: double, count: double, dtadfile: string, visapost: string, occup: string, entdepa: string, entdepd: string, entdepu: string, matflag: string, biryear: double, dtaddto: string, gender: string, insnum: string, airline: string, admnum: double, fltno: string, visatype: string]>

##### Convert Data Types

In [4]:
df_spark = df_spark.withColumn("cicid", df_spark["cicid"].cast(IntegerType())).withColumn("i94cit", df_spark["i94cit"].cast(IntegerType())).withColumn("i94res", df_spark["i94res"].cast(IntegerType())).withColumn("i94mode", df_spark["i94mode"].cast(IntegerType())).withColumn("i94bir", df_spark["i94bir"].cast(IntegerType())).withColumn("i94visa", df_spark["i94visa"].cast(IntegerType())).withColumn("biryear", df_spark["biryear"].cast(IntegerType()))
df_spark.printSchema

<bound method DataFrame.printSchema of DataFrame[cicid: int, i94yr: double, i94mon: double, i94cit: int, i94res: int, i94port: string, arrdate: double, i94mode: int, i94addr: string, depdate: double, i94bir: int, i94visa: int, count: double, dtadfile: string, visapost: string, occup: string, entdepa: string, entdepd: string, entdepu: string, matflag: string, biryear: int, dtaddto: string, gender: string, insnum: string, airline: string, admnum: double, fltno: string, visatype: string]>

##### Rename Columns and Remove Unneeded Columns

In [5]:
df_spark = df_spark.select(col('cicid').alias('immigration_id'),col('i94cit').alias('citizenship_country_id'),col('i94res').alias('residency_country_id'),col('i94port').alias('arrival_city_id'),col('arrdate').alias('arrival_date'),col('i94mode').alias('travel_mode_id'),col('i94addr').alias('current_state_id'),col('depdate').alias('departure_date'),col('i94bir').alias('immigration_age'),col('i94visa').alias('visa_type_id'),col('biryear').alias('birth_year'),col('gender'))


In [6]:
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(float(x))).isoformat() if x else None)

In [7]:
df_spark = df_spark.withColumn("arrival_date", get_date(df_spark.arrival_date))

In [8]:
df_spark = df_spark.withColumn("departure_date", get_date(df_spark.departure_date))

In [9]:
df_spark = df_spark.withColumn("arrival_date", to_date(df_spark.arrival_date, 'yyyy-MM-dd')).withColumn("departure_date", to_date(df_spark.departure_date, 'yyyy-MM-dd'))


In [170]:
#df_spark.take(5)
#df_spark.describe("biryear").show()
#df_spark.count()
#df_spark.select("cicid").distinct().show()
df_spark.show()

+--------------+----------------------+--------------------+---------------+------------+--------------+----------------+--------------+---------------+------------+----------+------+
|immigration_id|citizenship_country_id|residency_country_id|arrival_city_id|arrival_date|travel_mode_id|current_state_id|departure_date|immigration_age|visa_type_id|birth_year|gender|
+--------------+----------------------+--------------------+---------------+------------+--------------+----------------+--------------+---------------+------------+----------+------+
|       5748517|                   245|                 438|            LOS|  2016-04-30|             1|              CA|    2016-05-08|             40|           1|      1976|     F|
|       5748518|                   245|                 438|            LOS|  2016-04-30|             1|              NV|    2016-05-17|             32|           1|      1984|     F|
|       5748519|                   245|                 438|            LOS|  20

In [10]:
df_spark1 = df_spark.select(col('arrival_date').alias('date_id')).dropDuplicates() 
df_spark2 = df_spark.select(col('departure_date').alias('date_id')).dropDuplicates() 


In [11]:
time_df = df_spark1.union(df_spark2).distinct()

In [12]:
time_df.count()

236

In [178]:
time_df.show()

+----------+
|   date_id|
+----------+
|2016-03-01|
|2016-04-25|
|2016-05-03|
|2016-08-15|
|2016-08-31|
|2016-07-26|
|2016-07-17|
|2016-08-23|
|2016-07-03|
|2016-05-26|
|2016-06-02|
|2016-02-04|
|2016-01-13|
|2016-04-22|
|2016-06-16|
|2069-05-03|
|2016-05-09|
|2016-07-19|
|2016-09-15|
|2016-05-23|
+----------+
only showing top 20 rows



In [171]:
df_spark.printSchema

<bound method DataFrame.printSchema of DataFrame[immigration_id: int, citizenship_country_id: int, residency_country_id: int, arrival_city_id: string, arrival_date: date, travel_mode_id: int, current_state_id: string, departure_date: date, immigration_age: int, visa_type_id: int, birth_year: int, gender: string]>

#### Demographic Data Cleanup

In [121]:
df_spark=spark.read.option("header", True).options(delimiter=';').csv("us-cities-demographics.csv")
df_spark.printSchema

<bound method DataFrame.printSchema of DataFrame[City: string, State: string, Median Age: string, Male Population: string, Female Population: string, Total Population: string, Number of Veterans: string, Foreign-born: string, Average Household Size: string, State Code: string, Race: string, Count: string]>

In [116]:
df_spark.select("Avg_household_size").distinct().show()

+------------------+
|Avg_household_size|
+------------------+
|              2.43|
|              3.89|
|              4.13|
|               2.6|
|              2.78|
|              3.04|
|              2.49|
|              3.19|
|              3.46|
|              3.02|
|              3.61|
|              3.06|
|              2.91|
|              3.44|
|              2.35|
|               2.7|
|              2.98|
|              3.13|
|              3.37|
|              3.24|
+------------------+
only showing top 20 rows



##### Convert Data Types

In [122]:
df_spark = df_spark.withColumn("Median Age", df_spark["Median Age"].cast(DoubleType())).withColumn("Male Population", df_spark["Male Population"].cast(IntegerType())).withColumn("Female Population", df_spark["Female Population"].cast(IntegerType())).withColumn("Total Population", df_spark["Total Population"].cast(IntegerType())).withColumn("Number of Veterans", df_spark["Number of Veterans"].cast(IntegerType())).withColumn("Average Household Size", df_spark["Average Household Size"].cast(DoubleType())).withColumn("Count", df_spark["Count"].cast(IntegerType()))
df_spark.printSchema


<bound method DataFrame.printSchema of DataFrame[City: string, State: string, Median Age: double, Male Population: int, Female Population: int, Total Population: int, Number of Veterans: int, Foreign-born: string, Average Household Size: double, State Code: string, Race: string, Count: int]>

##### Rename fields, sort by city, state, and race count, and then drop duplicates based on city & state

In [123]:
df_spark = df_spark.select(col('City').alias('city'),col('State Code').alias('state_code'),col('Median Age').alias('median_age'),col('Male Population').alias('male_population'),col('Female Population').alias('female_population'),col('Total Population').alias('total_population'),col('Number of Veterans').alias('veteran_population'),col('Foreign-born').alias('foreign_population'),col('Average Household Size').alias('avg_household_size'),col('Race').alias('race_majority'),col('Count').alias('race_count')).orderBy(col('city').asc(),col('state_code').asc(),col('race_count').desc()).dropDuplicates(['city','state_code'])


In [124]:
df_spark.show()

+--------------+----------+----------+---------------+-----------------+----------------+------------------+------------------+------------------+------------------+----------+
|          city|state_code|median_age|male_population|female_population|total_population|veteran_population|foreign_population|avg_household_size|     race_majority|race_count|
+--------------+----------+----------+---------------+-----------------+----------------+------------------+------------------+------------------+------------------+----------+
|     Asheville|        NC|      37.9|          42100|            46407|           88507|              4973|              6630|              2.18|             White|     77979|
|          Mesa|        AZ|      36.9|         234998|           236835|          471833|             31808|             57492|              2.68|             White|    413010|
|       Norwalk|        CA|      35.0|          53125|            54020|          107145|              3027|       

In [113]:
df_spark.count()

596

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.